# Data Preparation and Cleaning

## 1. SETUP & IMPORTS

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)

## 2. DATEN LADEN

In [ ]:
base_dir = Path.cwd()
data_file = 'OECD.WISE.WDP,DSD_HSL@DF_HSL_CWB,+all.csv'
candidates = [
    base_dir / 'data' / data_file,
    base_dir.parent / 'data' / data_file,
]
data_path = next((p for p in candidates if p.exists()), None)
if data_path is None:
    raise FileNotFoundError(f"Datei nicht gefunden. Versucht wurden: {candidates}")

print(f"Lade Daten von: {data_path}")

try:
    df = pd.read_csv(data_path)
    print("Daten erfolgreich geladen!")
except FileNotFoundError:
    print("FEHLER: Datei nicht gefunden. Prüfe den Pfad oder verschiebe die CSV in einen 'data' Ordner.")


## 3. ERSTE INSPEKTION

In [ ]:
print(f"Dimensionen: {df.shape}")
print(df.head(3))
print(df.info())

## 4. DATEN BEREINIGUNG

In [ ]:
cols_to_drop = [
    'STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'ACTION', 
    'REF_AREA',      # Wir haben 'Reference area'
    'MEASURE',       # Wir haben 'Measure'
    'UNIT_MEASURE',  # Wir haben 'Unit of measure'
    'AGE',           # Wir haben 'Age'
    'SEX',           # Wir haben 'Sex'
    'EDUCATION_LEV', # Wir haben 'Education level'
    'DOMAIN',        # Wir haben 'Domain'
    'OBS_STATUS', 'Observation status', 
    'UNIT_MULT', 'Unit multiplier', 
    'DECIMALS', 'Decimals', 
    'BASE_PER', 'Base period',
    'Observation value', # War leer, wir nutzen OBS_VALUE
    'Time period' # Ist oft leer oder doppelt, wir nutzen TIME_PERIOD
]

# Nur existierende Spalten droppen
df_clean = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

# Spaltennamen normalisieren
df_clean.columns = [c.lower().replace(' ', '_') for c in df_clean.columns]
df_clean.rename(columns={'obs_value': 'value', 'time_period': 'year'}, inplace=True)

print("\n--- Spalten nach Bereinigung ---")
print(df_clean.columns.tolist())

## 5. UMGANG MIT FILTERN & DUPLIKATEN

In [ ]:
print("\nVerfügbare Measures (Top 10):")
print(df_clean['measure'].unique()[:10])

In [ ]:
# Sortieren nach Jahr, damit 'last' wirklich das aktuellste ist
df_clean = df_clean.sort_values('year')

#Einheiten definieren
group_cols = ['reference_area', 'measure', 'sex', 'age', 'education_level', 'domain']

df_time = df_clean.copy()

#Filtern auf 1 Jahr, um Unabhängigkeit zu wahren
df_latest = df_clean.drop_duplicates(subset=group_cols, keep='last')

print(f"Datensatz für Zeitreihen (df_time): {len(df_time)} Zeilen")
print(f"Datensatz für Hypothesentests (df_latest): {len(df_latest)} Zeilen")

## 6. MISSING VALUES HANDLING

In [ ]:
print("Analyse fehlender Werte vor dem Bereinigen:")

missing_counts = df_clean.isnull().sum()
print(missing_counts[missing_counts > 0])

# Welche Länder haben am meisten Datenlücken?
missing_by_country = df_clean.isnull().groupby(df_clean['reference_area']).sum().sum(axis=1).sort_values(ascending=False)

# Nur plotten, wenn es tatsächlich fehlende Werte gibt
if missing_by_country[missing_by_country > 0].shape[0] > 0:
    plt.figure(figsize=(12, 6))
    missing_by_country[missing_by_country > 0].head(10).plot(kind='bar', color='salmon')
    plt.title("Top 10 Länder mit den meisten fehlenden Werten")
    plt.ylabel("Anzahl fehlender Datenpunkte")
    plt.show()
else:
    print("Keine fehlenden Werte nach Ländern gefunden.")

# Beide Datensätze von leeren Werten in der Zielvariable bereinigen
df_time = df_time.dropna(subset=['value'])
df_latest = df_latest.dropna(subset=['value'])



### Analyse der fehlenden Werte:

Der Output zeigt, dass in den für die Analyse relevanten Spalten keine fehlenden Werte (Missing Values) vorhanden sind. Die missing_counts Series ist leer.

Da der Datensatz vollständig ist (0% Missingness), entfällt die Notwendigkeit, zwischen den Mechanismen MCAR (Missing Completely At Random), MAR (Missing At Random) oder MNAR (Missing Not At Random) zu unterscheiden.

=> Es liegt kein Bias durch fehlende Daten vor (z. B. dass ärmere Länder seltener Daten berichten würden = MNAR). Wir können dropna() formal ausführen (es löscht nichts) und mit dem vollständigen Datensatz weiterarbeiten, was die interne Validität der Studie stärkt.

Trtozdem könnte bei uns ein **Sampling** oder **Selection Bias** auftreten. 

Theoretisch wollen wir Aussagen über den Zusammenhang von Wohlstand und Sicherheit für *alle* Länder treffen (Population). Unser Datensatz enthält jedoch fast ausschließlich **OECD-Länder**. Die OECD besteht primär aus westlichen Industrienationen mit hohem Einkommen und stabilen demokratischen Strukturen. Länder des globalen Südens, Schwellenländer und Krisenregionen sind stark unterrepräsentiert.

Unsere Ergebnisse leiden somit unter einem **Selection Bias**. Gefundene Korrelationen (z.B. "Geld macht sicher") gelten streng genommen nur für den Kontext entwickelter Industrienationen ("Deckeneffekt" des Sicherheitsgefühls). **Eine Generalisierung auf die gesamte Weltbevölkerung ist statistisch nicht zulässig.**

## 7. SPEICHERN

In [ ]:


df_time.to_csv(base_dir.parent / 'data' / 'oecd_full_time_series.csv', index=False)
df_latest.to_csv(base_dir.parent / 'data' / 'oecd_snapshot_latest.csv', index=False)

print("Beide Datensätze gespeichert.")

Für die Analyse haben wir **zwei Versionen des Datensatzes** erstellt:

**oecd_full_time_series.csv** : Enthält alle historischen Daten. Wird verwendet für deskriptive Zeitreihenanalysen (Trendentwicklung).

**oecd_snapshot_latest.csv** : Enthält nur den jeweils aktuellsten Datenpunkt pro Land und Kategorie. Wird verwendet für induktive Statistik (Hypothesentests, Korrelationen), um die statistische Annahme der Unabhängigkeit der Beobachtungen zu wahren und Pseudoreplikation durch wiederholte Messungen desselben Landes zu vermeiden.